In [ ]:
!pip install -q jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user
!jupyter nbextension enable hinterland/hinterland

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

In [1]:
pip install -qU pytorch-tabnet scikit-learn neptune

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.9/487.9 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━

In [1]:
!nvidia-smi

Fri Aug  1 12:46:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub


arbaazbeg_bitgrit_air_pollution_path = kagglehub.dataset_download('arbaazbeg/bitgrit-air-pollution')
print('Data source import complete.')

Data source import complete.


In [3]:
import torch


def print_config(config):
    attrs = {}
    attrs.update(config.__dict__)

    for key, value in config.__class__.__dict__.items():
        if not key.startswith('__') and key not in attrs:
            attrs[key] = value

    for key, value in attrs.items():
        print(f"{key}: {value}")


class Config:
    # Model
    model_name = "tabnet.month1.only.repro"
    input_dim = 6
    target_dim = 1

    # Device & reproducibility
    device = "cuda" if torch.cuda.is_available() else "cpu"
    seed = 5274

    # Optimization
    optimizer_name = "AdamW"
    lr = 1e-4
    weight_decay = 1e-3

    grad_accum = False
    grad_accum_steps = 1
    batch_size = 32  # // grad_accum_steps

    # Training schedule
    num_epochs = 150
    scheduler_name = "default"

    # Regularization
    dropout = 0.0
    drop_path_rate = 0.0
    label_smoothing = 0.0

    # Experiment tracking
    neptune_token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlOGE2YjNiZS1mZGUyLTRjYjItYTg5Yy1mZWJkZTIzNzE1NmIifQ=="
    with_id = ""
    resume = False

config = Config()

In [4]:
import random
import numpy as np
import torch

def setup_reproducibility(config):
    random.seed(config.seed)
    np.random.seed(config.seed)
    torch.manual_seed(config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(config.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False, warn_only=True)
    torch.set_float32_matmul_precision("high")

setup_reproducibility(config)

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


def load_and_scale_data(path, SEED):
    data = pd.read_csv(path).drop(columns=['id'])
    data.dropna(inplace=True)
    data = data[data["month"] == 1]
    data = data.to_numpy()#.astype(np.float32)
    
    inputs = data[:, :-1]
    targets = data[:, -1].reshape(-1, 1)
    
    scaler = StandardScaler()
    inputs = scaler.fit_transform(inputs)
    
    train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
        inputs, 
        targets, 
        train_size=0.8, 
        random_state=SEED, 
        shuffle=False
    )

    return (
        train_inputs.astype(np.float32),
        train_targets.astype(np.float32),
        eval_inputs.astype(np.float32),
        eval_targets.astype(np.float32)
    )

path = arbaazbeg_bitgrit_air_pollution_path+"/train.csv"
data = load_and_scale_data(path, config.seed)
mean = data[1].mean()
std = data[1].std()
mean, std, len(data[0]), data[1].shape

(32.116894, 71.55909, 267, (267, 1))

In [6]:
from torch.utils.data import Dataset


class TabularDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets
        assert len(self.inputs) == len(self.targets), "len not eq"

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        return self.inputs[index], self.targets[index]

train_ds = TabularDataset(data[0], data[1])
eval_ds = TabularDataset(data[2], data[3])    

In [7]:
from torch.utils.data import DataLoader


def build_loader(
    SEED,
    ds,
    train=True,
    batch_size=16,
    shuffle=False,
    num_workers=4,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
):
    def seed_worker(worker_id):
        worker_seed = torch.initial_seed() % 2**32
        np.random.seed(worker_seed)
        random.seed(worker_seed)

    generator = torch.Generator()
    generator.manual_seed(SEED if train else SEED+1)

    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=pin_memory,
        drop_last=drop_last,
        persistent_workers=persistent_workers,
        worker_init_fn=seed_worker,
        generator=generator,
        #sampler=DistributedSampler(
        #    train_ds,
        #    shuffle=True,
        #    drop_last=True,
        #    seed=config.seed
        #)
    )
    
    
train_dl = build_loader(
    config.seed,
    train_ds,
    train=True,
    batch_size=config.batch_size,
    shuffle=True,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

eval_dl = build_loader(
    config.seed,
    eval_ds,
    train=False,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=0,
    drop_last=True,
    pin_memory=True,
    persistent_workers=False,
)

In [8]:
import torch.nn as nn
from pytorch_tabnet.tab_network import TabNet
from pytorch_tabnet.utils import create_group_matrix


class Model(nn.Module):
    def __init__(self, mean, std):
        super().__init__()
        
        grp_list = [[i] for i in range(6)]
        group_matrix = create_group_matrix(
            grp_list,
            config.input_dim,
        ).to(config.device)

        self.backbone = TabNet(
            input_dim=config.input_dim,
            output_dim=config.target_dim,
            group_attention_matrix=group_matrix
        ).to(config.device)

        self.register_buffer('mean', torch.tensor(mean))
        self.register_buffer('std',  torch.tensor(std))

    def forward(self, x):
        x = self.backbone(x)[0]
        return (x * self.std) + self.mean
        
model = Model(mean, std).to(config.device)       
        
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.lr,
    weight_decay=config.weight_decay,
    fused=True
)

In [9]:
from transformers import get_cosine_schedule_with_warmup


total_training_steps = (len(train_dl)) * config.num_epochs
warmup_steps = int(total_training_steps * 0.05)

scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_training_steps
)

In [10]:
from collections import OrderedDict


def remove_orig_mod(state_dict):
    new_state_dict = OrderedDict()
    prefix = "_orig_mod."
    for key, value in state_dict.items():
        if key.startswith(prefix):
            new_key = key[len(prefix):]
        else:
            new_key = key
        new_state_dict[new_key] = value
    return new_state_dict

In [11]:
import neptune

def setup_neptune(config):
    if not config.resume:
        neptune_run = neptune.init_run(
            project="arbaaz/bitgrit-air-pollution-1",
            name=config.model_name,
            api_token=config.neptune_token
        )

        neptune_run["h_parameters"] = {
            "model_name": config.model_name,
            "optimizer_name": config.optimizer_name,
            "learning_rate": config.lr,
            "scheduler_name": config.scheduler_name,
            "weight_decay": config.weight_decay,
            "dropout": config.dropout,
            "label_smoothing": config.label_smoothing,
            "num_epochs": config.num_epochs,
            "batch_size": config.batch_size,
            "grad_accum_steps": config.grad_accum_steps,
        }
    else:
        neptune_run = neptune.init_run(
            project="arbaaz/onward-speed-and-structure",
            with_id=config.with_id,
            api_token=config.neptune_token
        )

    return neptune_run

neptune_run = setup_neptune(config)    

[neptune] [warning] NeptuneWarning: By default, these monitoring options are disabled in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', 'capture_hardware_metrics'. You can set them to 'True' when initializing the run and the monitoring will continue until you call run.stop() or the kernel stops. NOTE: To track the source files, pass their paths to the 'source_code' argument. For help, see: https://docs-legacy.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/arbaaz/bitgrit-air-pollution-1/e/BIT-28


In [12]:
import os
import torch.nn.functional as F
from sklearn.metrics import root_mean_squared_error
    

def loss_fn(preds, targets):
    return F.mse_loss(preds, targets)


def metric_fn(preds, targets):
    mse = F.mse_loss(preds, targets)
    rmse = -(torch.sqrt(mse) / 100)
    return torch.exp(rmse)


def metric_fn_np(preds, targets):
    rmse = root_mean_squared_error(preds, targets)
    rmse = -(rmse / 100)
    return np.exp(rmse)

In [13]:
from tqdm.auto import tqdm
            

def return_logits(model, inputs):
    return  model(inputs)


def train_step(model, inputs, targets):
    with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
        logits = return_logits(model, inputs)
        mse = loss_fn(logits, targets)
        rmse = metric_fn(logits, targets)
    return mse, rmse


def eval_step(model, inputs, targets):
    with torch.inference_mode():
        with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
            logits = return_logits(model, inputs)
            mse = loss_fn(logits, targets)
            rmse = metric_fn(logits, targets)
    return mse, rmse


def inference_step(model, inputs):
    model.eval() # FIX THIS 
    with torch.inference_mode():
        with torch.autocast(device_type=config.device, dtype=torch.float16, cache_enabled=True):
            logits = return_logits(model, inputs)
    return logits
    

def backward_step(loss, optimizer, scaler, scheduler):
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()
    scheduler.step()
          
    
def reduce_outputs(outputs):
    data = {}
    for key in outputs[0].keys():
        if outputs[0][key].dim() == 0:
            value = torch.stack([x[key] for x in outputs]).mean().item()
            data[key] = value
    return data


def save_ckpt(model, metadata, ckpt_path):
    data = {"state_dict": model.state_dict()}
    for k, v in metadata.items():
        data[k] = v
    torch.save(data, ckpt_path)


def log_and_print(neptune_run, data, log=True, print=True):
    if log:
        for key, value in data.items():
            neptune_run[key].append(value)
        
    if print:
        tqdm.write((
            f"Epoch: {data['epoch']}, "
            f"train/mse: {data['train/mse']:.4f}, "
            f"eval/mse: {data['eval/mse']:.4f}, "
            f"train/rmse: {data['train/rmse']:.4f}, "
            f"eval/rmse: {data['eval/rmse']:.4f}, "
        ))

In [14]:
def train_one_epoch(
    model, 
    optimizer,
    scaler,  
    scheduler,
    train_dl,
    epoch,
    num_epochs,
    resume_epoch,
    resume, 
    log_lr_on_step,
    neptune_run
):
    model.train()
    pbar = tqdm(
        train_dl,
        desc=f"Epoch: {epoch}/{num_epochs}",
        leave=False,
        dynamic_ncols=True,
        colour="red",
        position=1,
    )
    outputs = []

    for index, (inputs, targets) in enumerate(pbar):
        if resume and epoch < resume_epoch:
            continue
        
        inputs = inputs.to(config.device, non_blocking=True)
        targets = targets.to(config.device, non_blocking=True)
    
        mse, rmse = train_step(model, inputs, targets)
        backward_step(mse, optimizer, scaler, scheduler)

        outputs.append({
            "train/mse": mse.detach().cpu(),
            "train/rmse": rmse.detach().cpu(),
        })

        if log_lr_on_step:
            log_and_print(
                neptune_run,
                data={"STEP/lr": scheduler.get_last_lr()[0]}, 
                log=True, 
                print=False
            )
            
    return outputs
            
            
def evaluate(model, eval_dl):
    model.eval()
    pbar = tqdm(
        eval_dl,
        desc=f"Evaluating",
        leave=False,
        dynamic_ncols=True,
        colour="blue",
        position=1,
    )
    outputs = []
    
    for inputs, targets in pbar:
        inputs = inputs.to(config.device, non_blocking=True)
        targets = targets.to(config.device, non_blocking=True)
        mse, rmse = eval_step(model, inputs, targets)
        
        outputs.append({
            "eval/mse": mse.detach().cpu(),
            "eval/rmse": rmse.detach().cpu(),
        })
        
    return outputs

In [15]:
def train(
    model, 
    optimizer, 
    scheduler, 
    train_dl,
    eval_dl, 
    num_epochs, 
    resume_epoch,
    resume,
    log_lr_on_step,
    neptune_run
):
    os.makedirs("/ckpts", exist_ok=True)
    train_outputs = []
    eval_outputs = []
    RMSE = float('-inf')
    
    scaler = torch.GradScaler()
    pbar = tqdm(range(0, num_epochs), position=0, colour="green")
    
    for epoch in pbar:
        train_outputs = train_one_epoch(
            model,
            optimizer, 
            scaler, 
            scheduler, 
            train_dl,
            epoch,
            num_epochs,
            resume_epoch,
            resume,
            log_lr_on_step,
            neptune_run
        )
        eval_outputs = evaluate(model, eval_dl)
        
        train_outputs = reduce_outputs(train_outputs)
        eval_outputs = reduce_outputs(eval_outputs)
        
        outputs = train_outputs | eval_outputs
        outputs["epoch"] = epoch
        log_and_print(neptune_run, outputs)

        if outputs["eval/rmse"] > RMSE:
            RMSE = outputs["eval/rmse"]
            metadata = {
                "train_rmse": outputs["train/rmse"],
                "eval_rmse": RMSE,
                "epoch": epoch
            }
            save_ckpt(model, metadata, f"/ckpts/{config.model_name}.pt")
            
    neptune_run.stop()
   
try:
    train(
        model, 
        optimizer, 
        scheduler, 
        train_dl,
        eval_dl, 
        num_epochs=config.num_epochs, 
        resume_epoch=0,
        resume=config.resume,
        log_lr_on_step=True,
        neptune_run=neptune_run
    )
except Exception:
    pass  # Silently ignore any errors
finally:
    neptune_run.stop()

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch: 0/150:   0%|          | 0/8 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 0, train/mse: 7489.2988, eval/mse: 2801.8059, train/rmse: 0.5008, eval/rmse: 0.6068, 


Epoch: 1/150:   0%|          | 0/8 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 148, train/mse: 5258.9780, eval/mse: 1511.8906, train/rmse: 0.5948, eval/rmse: 0.7105, 


Epoch: 149/150:   0%|          | 0/8 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch: 149, train/mse: 6472.4600, eval/mse: 1524.8649, train/rmse: 0.5718, eval/rmse: 0.7096, 
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Waiting for the remaining 247 operations to synchronize with Neptune. Do not kill this process.
[neptune] [info   ] All 247 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/arbaaz/bitgrit-air-pollution-1/e/BIT-28/metadata


In [64]:
import pandas as pd

path_test = arbaazbeg_bitgrit_air_pollution_path+"/test.csv"
test = pd.read_csv(path_test)
ids = test["id"]
test = test.drop(columns=['id'])
test

,latitude,longitude,day_of_year,day_of_week,hour,month
0,-42.343,147.372,31,4,15,1
1,37.053,127.406,28,1,12,1
2,49.105,-123.189,14,1,2,1
3,42.014,12.774,10,4,3,1
4,47.778,13.002,31,4,11,1
...,...,...,...,...,...,...
2734,37.216,126.748,28,1,12,1
2735,48.368,14.338,31,4,11,1
2736,48.139,16.940,31,4,11,1
2737,-27.357,153.083,31,4,12,1


In [ ]:
def load_and_scale_data(path, SEED):
    data = pd.read_csv(path).drop(columns=['id'])
    data.dropna(inplace=True)
    data = data[data["month"] == 1]
    data = data.to_numpy()#.astype(np.float32)
    
    inputs = data[:, :-1]
    targets = data[:, -1].reshape(-1, 1)
    
    scaler = StandardScaler()
    inputs = scaler.fit_transform(inputs)

In [65]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_df = pd.read_csv(path).drop(columns=['id'])
train_df.dropna(inplace=True)
train_df = train_df[train_df["month"] == 1]

train_df = train_df.to_numpy()
train_inputs = train_df[:, :-1]
train_inputs = scaler.fit_transform(train_inputs)


In [66]:
from sklearn.preprocessing import StandardScaler

scaler_test = StandardScaler()
test = test.to_numpy()
test = scaler.transform(test)
test = torch.tensor(test).float()
test.shape

torch.Size([2739, 6])

In [67]:
test

tensor([[-4.5653,  2.5152,  1.0443,  1.4761, -0.1017,  0.0000],
        [-0.1742,  2.1847,  0.7734, -0.1858, -0.5244,  0.0000],
        [ 0.4923, -1.9634, -0.4907, -0.1858, -1.9336,  0.0000],
        ...,
        [ 0.4389,  0.3562,  1.0443,  1.4761, -0.6654,  0.0000],
        [-3.7365,  2.6098,  1.0443,  1.4761, -0.5244,  0.0000],
        [-0.1664,  2.1785,  0.7734, -0.1858, -0.5244,  0.0000]])

In [46]:
test

tensor([[-3.3579,  1.6801,  0.4517,  0.5812,  1.2741, -0.0191],
        [ 0.0119,  1.4056, -0.0724, -1.6075,  0.3369, -0.0191],
        [ 0.5235, -2.0393, -2.5181, -1.6075, -2.7869, -0.0191],
        ...,
        [ 0.4825, -0.1129,  0.4517,  0.5812,  0.0245, -0.0191],
        [-2.7219,  1.7586,  0.4517,  0.5812,  0.3369, -0.0191],
        [ 0.0180,  1.4004, -0.0724, -1.6075,  0.3369, -0.0191]],
       device='cuda:0')

In [68]:
import os

os.listdir("/ckpts/")

['tabnet.month1.only.pt']

In [69]:
ckpt_path = f"/ckpts/{config.model_name}.pt"
ckpt = torch.load(ckpt_path)
ckpt["train_rmse"], ckpt["eval_rmse"], ckpt["epoch"]

(0.61403489112854, 0.731635332107544, 138)

In [70]:
model.load_state_dict(ckpt["state_dict"])
#[param.device for param in model.parameters()]

<All keys matched successfully>

In [71]:
test = test.cuda()
logits = inference_step(model, inputs=test)
logits.shape

torch.Size([2739, 1])

In [72]:
logits_ = logits.cpu().detach().double().numpy()
logits_.shape

(2739, 1)

In [73]:
if np.isnan(logits_).any():
    print(np.isnan(logits_.sum()))
    logits_ = np.nan_to_num(logits_, nan=0.0)

In [74]:
(logits_ < 0).any(), (logits_ < 0).sum()

(True, 30)

In [75]:
logits_.mean(), logits_.std()

(73.18010108616284, 425.13476653587054)

In [76]:
df_pred = pd.DataFrame(ids)
df_pred

,id
0,0
1,1
2,2
3,3
4,4
...,...
2734,2734
2735,2735
2736,2736
2737,2737


In [77]:
df_pred["pollution_value"] = logits_
df_pred

,id,pollution_value
0,0,168.875000
1,1,24.328125
2,2,-7.406250
3,3,62.343750
4,4,102.125000
...,...,...
2734,2734,23.781250
2735,2735,94.562500
2736,2736,76.875000
2737,2737,103.562500


In [78]:
#config.model_name = "tabnet.100ep.wd.1e3.150ep.relu.fixed"
config.model_name

'tabnet.month1.only'

In [79]:
df_pred.to_csv(f"{config.model_name}.csv", index=False)